In [1]:
from numpy import *
from numpy.random import *
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PlotFuncs import *
from LabFuncs import *
from Params import *
from HaloFuncs import *
from scipy.stats import norm
import pandas


n = 300
v = linspace(0.01,800.0,n)

# Halo params
HaloModel = SHMpp
v0 = HaloModel.RotationSpeed
v_esc = HaloModel.EscapeSpeed
sig_beta = HaloModel.SausageDispersionTensor
sig_iso = array([1.0,1.0,1.0])*v0/sqrt(2.0)
day = 67.0 

# Load shards
df = pandas.read_csv('../data/FitShards_red.csv')
names = df.group_id
nshards = size(names)
velocities = zeros(shape=(nshards,3))
dispersions = zeros(shape=(nshards,3))
velocities[0:(nshards),0] = df.vx # stream velocities
velocities[0:(nshards),1] = df.vy
velocities[0:(nshards),2] = df.vz
dispersions[0:(nshards),0] = df.sigx # dispersion tensors
dispersions[0:(nshards),1] = df.sigy
dispersions[0:(nshards),2] = df.sigz
pops = df.population
Psun = df.Psun
weights = ShardsWeights(names,pops,Psun)


# Smooth Speed distributions
fv_Iso_lab = SpeedDist_Triaxial(v,day,sig_iso) # Round halo
fv_Saus_lab = SpeedDist_Triaxial(v,day,sig_beta) # Sausage
fv_Iso_gf = SpeedDist_Triaxial(v,day,sig_iso,GravFocus=True) # Round halo
fv_Saus_gf = SpeedDist_Triaxial(v,day,sig_beta,GravFocus=True) # Sausage
fv_lab = zeros(shape=(nshards,n))
fv_gf = zeros(shape=(nshards,n))

for i in range(0,nshards):
    v_s = velocities[i,:]
    sig_s = dispersions[i,:]
    fv_lab[i] = weights[i]*SpeedDist_Triaxial(v,day,sig_s,v_shift=v_s,EscapeSpeed=False)
    

/Users/ciaranohare/Work/Zaragoza/HaloSpin/code/HaloFuncs.py:53: RuntimeWarning: invalid value encountered in double_scalars
  *erfi(v_esc/(sqrt(2)*sigr)*sqrt(beta/(1-beta)))



In [ ]:
n_xi = 10
xi_max = 0.1
xi_vals = linspace(0.0,xi_max,n_xi)

fvs_lab = sum(fv_lab,0)

xi_S2 = 0.9
fvnS2 = sum(fv_lab[3:,:],0)+fv_lab[0,:]
fvnS2 /= trapz(fvnS2,v)
fvS2 = fv_lab[1,:]+fv_lab[2,:]
fvS2 /= trapz(fvS2,v)
fvS2 = xi_S2*fvS2+(1-xi_S2)*fvnS2

def RSdef(fvs_lab):
    fv0 = (((1-eta_saus)*fv_Iso_lab + eta_saus*fv_Saus_lab))
    R0 = trapz(fv0**2.0/v,v)**(-0.25)
    RS = zeros(shape=n_xi)
    for i in range(0,n_xi):
        xi = xi_vals[i]
        fv1 = (((1-eta_saus-xi)*fv_Iso_lab + eta_saus*fv_Saus_lab + xi*fvs_lab))
        R1 = trapz(fv1**2.0/v,v)**(-0.25)
        RS[i] = 100*(1-R1/R0)

print(RSdef(fvs_lab))
print(RSdef(fvS2))